## Projecto K-NN

KNN es un algoritmo de aprendizaje supervisado que clasifica o predice valores basándose en la cercanía de los datos de entrenamiento más similares a un nuevo dato, utilizando la mayoría de las etiquetas de los k ejemplos más cercanos.

*¿Qué es k?*

En el contexto del algoritmo k-neighbors, **k** representa el número de puntos más cercanos al que queremos medir, y que se utilizarán para realizar la predicción o clasificación para este nuevo dato.

En otras palabras, k determina cuántos puntos cercanos se tomarán en cuenta para tomar una decisión sobre la etiqueta o el valor de un nuevo dato. Por ejemplo, si k = 3, se tomarán en cuenta los 3 puntos más cercanos para clasificar o predecir el valor del nuevo dato.

*¿Qué es una etiqueta?*

En el contexto del aprendizaje supervisado, una etiqueta se refiere a la información conocida o el valor objetivo asociado a cada dato de entrenamiento. En otras palabras, es la respuesta deseada que el modelo intentará aprender durante el proceso de entrenamiento.

Por ejemplo, si estamos trabajando en un problema de **clasificación**, donde queremos que un modelo clasifique imágenes de animales en "perros", "gatos" o "aves", cada imagen tendrá una etiqueta asociada que indicará a qué categoría pertenece (por ejemplo, "perro" o "gato").

En el caso de un problema de **regresión**, donde queremos predecir un valor numérico, las etiquetas serían los valores numéricos correspondientes a cada ejemplo de entrenamiento.

### Sistema de recomendación de películas

¿Seríamos capaces de predecir qué películas podrían ser o no un éxito comercial?

## 1. Carga el conjunto de datos

In [1]:
#Import necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Example importing the CSV here

movies = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv')
credits = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv')


## 2. Creación de una base de datos

In [3]:
import sqlite3

# Conecta y crea una base de datos SQLite.
conn = sqlite3.connect('movie_database.db')

# Almacena los DataFrames en las tablas 'movies' y 'credits' de la base de datos.
movies.to_sql('movies', conn, index=False, if_exists='replace')
credits.to_sql('credits', conn, index=False, if_exists='replace')

# Ejecuta la unión de las tablas en SQL para obtener una tercera tabla con la información unificada.
query = '''
        SELECT m.*, c.*
        FROM movies m
        INNER JOIN credits c ON m.title = c.title
        '''

# Lee el resultado en un DataFrame.
result_df = pd.read_sql_query(query, conn)

# Cierra la conexión a la base de datos.
conn.close()

# Ahora 'result_df' contiene la información unificada de ambas tablas.

In [4]:
result_df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,title,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [5]:
# Lista de columnas que deseas mantener en el DataFrame.
columns_to_keep = ['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']

# Elimina las columnas que no están en la lista 'columns_to_keep'.
result_df = result_df.drop(columns = [col for col in result_df.columns if col not in columns_to_keep])

'''columns = []
for col in result_df.columns:
    if col not in columns_to_keep:
        columns.append(col)'''

'columns = []\nfor col in result_df.columns:\n    if col not in columns_to_keep:\n        columns.append(col)'

In [6]:
result_df.head()

,genres,keywords,overview,title,movie_id,title,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Avatar,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,Spectre,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,The Dark Knight Rises,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",John Carter,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


## 3. Transforma los datos

In [7]:
# Eliminamos la columna duplicada 'title'

result_df.columns.values[3] = 'todelete'
result_df = result_df.drop('todelete', axis=1)

In [8]:
result_df.head(4)

,genres,keywords,overview,movie_id,title,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."


In [9]:
result_df['crew'].iloc[0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [10]:
# De cada uno de los JSONs, reemplaza las columnas genres y keywords con su atributo name
# Para la columna cast, selecciona los tres primeros nombres.

import json

def load_json_safe(json_str, default_value=None): # la función load_json_safe carga una cadena de JSON en un objeto Python.
                                                  # json_str es la cadena JSON que se va a cargar
                                                  # default_vlue es el valor predeterminado que se devuelve si hay un arror al cargar la cadena JSON
    try:
        return json.loads(json_str)
    except (TypeError, json.JSONDecodeError):
        return default_value

# Transformar los datos de las columnas genres y keywords

generos_peliculas = []

for pelicula in result_df["genres"]:
  lista_generos = []
  for genero in json.loads(pelicula):
    lista_generos.append(genero['name'])
  generos_peliculas.append(lista_generos)
 #print(generos_peliculas)

result_df['genres'] = generos_peliculas

# result_df.head()


#result_df["genres"] = result_df["genres"].apply(lambda x: [item["name"] for item in json.loads(x)] if pd.notna(x) else None)

keywords_peliculas = []

for pelicula in result_df['keywords']:
  lista_keywords =[]
  for keyword in json.loads(pelicula):
    lista_keywords.append(keyword['name'])
  keywords_peliculas.append(lista_keywords)

result_df['keywords'] = keywords_peliculas

# result_df.head()

#result_df["keywords"] = result_df["keywords"].apply(lambda x: [item["name"] for item in json.loads(x)] if pd.notna(x) else None)

cast_peliculas = []

for pelicula in result_df['cast']:
  lista_cast = []
  for actor in json.loads(pelicula):
    lista_cast.append(actor['name'])
  cast_peliculas.append(lista_cast[:3])

result_df['cast'] = cast_peliculas

# result_df.head()

# Transformar la columna cast para seleccionar solo los tres primeros nombres
#result_df["cast"] = result_df["cast"].apply(lambda x: [item["name"] for item in json.loads(x)][:3] if pd.notna(x) else None)

director_peliculas = []

for pelicula in result_df['crew']:
    lista_director = []
    for director in json.loads(pelicula):
        if director['job'] == 'Director':
            lista_director.append(director['name'])
    director_peliculas.append(lista_director)


result_df['crew'] = director_peliculas

# result_df.head()

# Transformar la columna crew para obtener el nombre del director
#result_df['crew'] = result_df['crew'].apply(lambda x: ' '.join([crew_member['name'] for crew_member in load_json_safe(x) if crew_member['job'] == 'Director']))


# Transformar la columna overview para convertirla en una lista
result_df['overview'] = result_df['overview'].apply(lambda x: [x])

# Ver el DataFrame resultante
result_df.head()

,genres,keywords,overview,movie_id,title,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[In the 22nd century, a paraplegic Marine is d...",19995,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Captain Barbossa, long believed to be dead, h...",285,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",[A cryptic message from Bond’s past sends him ...,206647,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",[Following the death of District Attorney Harv...,49026,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[John Carter is a war-weary, former military c...",49529,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [11]:
# Convertir todos los elementos de las listas a cadenas antes de concatenarlos

result_df['overview'] = result_df['overview'].apply(lambda x: [str(x)])
# for i in range(len(result_df['overview'])):
    # result_df['overview'][i] = [str(result_df['overview'][i])]



result_df['genres'] = result_df['genres'].apply(lambda x: [str(genre) for genre in x])
result_df['keywords'] = result_df['keywords'].apply(lambda x: [str(keyword) for keyword in x])
result_df['cast'] = result_df['cast'].apply(lambda x: [str(actor) for actor in x])
result_df['crew'] = result_df['crew'].apply(lambda x: [str(crew_member) for crew_member in x])

# Combinar todas las columnas convertidas en una sola columna llamada 'tags'
result_df['tags'] = result_df['overview'] + result_df['genres'] + result_df['keywords'] + result_df['cast'] + result_df['crew']

# Convertir la lista de tags en una cadena separada por comas
result_df['tags'] = result_df['tags'].apply(lambda x: ','.join(x))

# Reemplazar las comas por espacios en blanco
result_df['tags'] = result_df['tags'].apply(lambda x: x.replace(",", " "))

# Eliminar las columnas individuales que ya hemos combinado
result_df.drop(columns=['genres', 'keywords', 'cast', 'crew', 'overview'], inplace=True)

# Ver el DataFrame resultante
result_df.iloc[0].tags

"['In the 22nd century  a paraplegic Marine is dispatched to the moon Pandora on a unique mission  but becomes torn between following orders and protecting an alien civilization.'] Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d Sam Worthington Zoe Saldana Sigourney Weaver James Cameron"

## 4. Crea modelo KNN

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer() # Creamos una instancia de la clase TfidfVectorizer, con la que haremos la transformación TF-IDF en los datos de entrada (tags).
tfidf_matrix = vectorizer.fit_transform(result_df['tags'])  # Se llama al método fit_transform() del objeto vectorizer para realizar dos pasos:
                                                            # 1. Ajuste (fitting): En esta etapa, el vectorizador "aprende" las características del conjunto
                                                            # de datos proporcionado (result_df['tags']).
                                                            # Analiza los textos de las etiquetas y determina el vocabulario único de palabras presentes
                                                            # en todas las etiquetas.
                                                            # 2. Transformación (transforming): Luego de haber ajustado el vectorizador,
                                                            # se aplica la transformación TF-IDF a los datos de entrada (result_df['tags']).
                                                            # Esto implica calcular los valores TF-IDF para cada palabra en las etiquetas de acuerdo
                                                            # con el vocabulario aprendido en el paso de ajuste.
                                                            # El resultado es una matriz numérica donde las filas representan las etiquetas
                                                            # y las columnas representan las palabras únicas del vocabulario,
                                                            # y los valores son los valores TF-IDF correspondientes.

## ¿Qué es TfidVectorizer?
TfidfVectorizer es una clase que se utiliza para transformar un conjunto de documentos de texto en una representación numérica llamada TF-IDF (Term Frequency-Inverse Document Frequency).

El proceso TF-IDF implica dos conceptos principales:

1. Frecuencia del término (TF): Mide la frecuencia de una palabra particular en un documento específico. Cuantas más veces aparezca una palabra en un documento, mayor será su valor de TF para ese documento.

2. Frecuencia inversa del documento (IDF): Mide la importancia de una palabra en el corpus total de documentos. Las palabras comunes que aparecen en muchos documentos tendrán un valor de IDF más bajo, mientras que las palabras menos comunes o distintivas tendrán un valor de IDF más alto.

La multiplicación de la TF y el IDF para cada palabra en un documento resulta en una representación numérica que captura la importancia relativa de las palabras en ese documento en comparación con el corpus general. Esto puede ser útil para tareas de análisis de texto, clasificación de documentos, agrupación de textos y otras aplicaciones de procesamiento de lenguaje natural.

TfidfVectorizer en scikit-learn toma un conjunto de documentos de texto y devuelve una matriz numérica donde las filas representan documentos y las columnas representan palabras, y los valores en la matriz son los valores TF-IDF correspondientes.

In [37]:
# Construir el modelo KNN con k=5 (5 vecinos más cercanos)

knn_model = NearestNeighbors(n_neighbors = 5, algorithm = 'brute', metric = 'cosine')
# n_neighbors = 5: Indica que el modelo debe encontrar los 5 vecinos más cercanos para cada punto en el espacio.
# algorithm = 'brute': Utiliza el enfoque "fuerza bruta" para encontrar los vecinos más cercanos.
# Esto significa que se compararán todos los puntos con todos los demás puntos en el espacio.
# metric = 'cosine': Utiliza la distancia coseno como métrica para medir la similitud entre puntos.
# En este caso, se utiliza la similitud coseno entre las representaciones TF-IDF.

knn_model.fit(tfidf_matrix) # Se ajusta el modelo k-NN utilizando la matriz TF-IDF que generaste previamente.
                            # Esto significa que el modelo aprenderá a encontrar los vecinos más cercanos en función de las representaciones TF-IDF
                            # y la métrica de similitud coseno.

# Función para obtener las recomendaciones de películas basadas en una película de entrada
def get_movie_recommendations(movie_title):
    movie_index = result_df[result_df['title'] == movie_title].index[0]
    # Dado un título de película, esta línea busca en el DataFrame el índice de la película que coincide con el título proporcionado.
    # El [0] al final extrae el índice del primer resultado encontrado.
    distances, indices = knn_model.kneighbors(tfidf_matrix[movie_index])
    # Utiliza el modelo k-NN previamente ajustado (knn_model) para encontrar los vecinos más cercanos a la película en el índice movie_index.
    # kneighbors devuelve dos matrices: distances (las distancias entre la película de consulta y sus vecinos cercanos)
    # e indices (los índices de las películas vecinas en la matriz TF-IDF).
    similar_movies = [(result_df['title'][i], distances[0][j]) for j, i in enumerate(indices[0])]
    #  Para cada índice de película en indices, crea una lista de tuplas que contiene el título de la película correspondiente
    # y la distancia calculada entre la película de consulta y la película vecina.
    # Esto es una forma de medir la similitud. La distancia más pequeña suele indicar una mayor similitud.
    return similar_movies[1:]  # Excluimos la película de entrada (la más cercana)

# Prueba del sistema de recomendación ingresando una película
input_movie = input("Escribe el título de una película: ") # Preguntamos al usuario por el título de una película
recommendations = get_movie_recommendations(input_movie) # Se llama a la función get_movie_recommendations con el título de la película proporcionado por el
                                                         # usuario. Esto devolverá la lista de recomendaciones.

print("Si te gustó '{}', te recomiendo que veas:".format(input_movie))
for movie, distance in recommendations:
    print("- {}".format(movie))

Escribe el título de una película: Inception
Si te gustó 'Inception', te recomiendo que veas:
- Hesher
- Don Jon
- Cypher
- The Helix... Loaded
